<a href="https://colab.research.google.com/github/alexmoed/MastersProject_Submission/blob/main/scripts/Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#If using colab mount drive first
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


 @brief ScanNet validation inference script with SONATA+PointGroup
 Organization and export logic assisted by Claude AI (Anthropic)
 Multiple prompts used for code structure, error handling, statistics tracking,
 and data processing pipeline organization (abbreviated from extended conversation).
 Base training framework and tools from: Pointcept Contributors (2023). Pointcept: A Codebase for Point Cloud Perception Research [online]. [Accessed 2025]. Available from: "https://github.com/Pointcept/Pointcept".# Original Author: Xiaoyang Wu (xiaoyang.wu.cs@gmail.com)

INSTALLS

In [ ]:
#Numpy versions are a ongoing issue in this install it must be below 1.25 to work 1.24.and 1.24.3 are tested and work
#Uninstall numpy first
#Restart session!!
!pip uninstall -y numpy
!pip uninstall -y numpy  # Run twice

# Find any installs of numpy left
!find /usr/local/lib/python*/site-packages -name "numpy*" -type d

# Delete ALL numpy folders
!rm -rf /usr/local/lib/python*/site-packages/numpy*
!rm -rf /usr/local/lib/python*/dist-packages/numpy*
!rm -rf ~/.local/lib/python*/site-packages/numpy*

# Clean pip cache too
!pip cache purge


!pip install numpy==1.24.0 --no-cache-dir

# Make sure the versions are correct and reload if not it needs to be 1.24.0
import numpy as np
if np.__version__ != "1.24.0":
   print(f"Wrong version: {np.__version__} Please restart runtime and rerun")


print(f"Numpy version: {np.__version__}")
print(f"Numpy location: {np.__file__}")


Found existing installation: numpy 1.24.0
Uninstalling numpy-1.24.0:
  Successfully uninstalled numpy-1.24.0
find: ‘/usr/local/lib/python*/site-packages’: No such file or directory
Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 315.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.22.0 requires numpy>=1.26.0, but you have numpy 1.24.0 which is incompatible.
contourpy 1.3.3 requires numpy>=1.25, but you have numpy 1.24.0 which is incompatible.
chex 0.1.90 requires numpy>=1.24.1, but you have numpy 1.24.0 which is incompatible.
jax 0.5.3 requires numpy>=1.25, but you have numpy 1.24.0 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.24.0 which is incompatible.
jaxlib 0.5.3 requires numpy>=1.25, but you have numpy 1.24.0 which is incomp

Numpy version: 1.24.0
Numpy location: /usr/local/lib/python3.11/dist-packages/numpy/__init__.py


In [ ]:
#Import universal dependencies that arent impacted by order
import os
import sys
import subprocess
import shutil
from pathlib import Path
from datetime import datetime
import gc
import json
import time
from collections import OrderedDict




This is a verison cuda install that after running once that it saves the download and caches the install for speed. Also handles torch installs

---



In [ ]:
# first batch of installs installing cuda and pytorch older versions: This will take 10+ minutes
%cd /content/drive/MyDrive/Pointcept/Installs
exec(open('/content/drive/MyDrive/Pointcept/Installs/setup_cuda_torch.py').read())


/content/drive/MyDrive/Pointcept/Installs
CUDA INSTALLATION

1. Cleaning existing CUDA installations...
✓ Cleaned

2. Installing CUDA keyring...
✓ Using cached keyring
✓ Keyring installed

3. Installing CUDA Toolkit 11.8...
✓ CUDA 11.8 installed

4. Setting environment variables...
✓ Environment set

5. Verifying installation...

✅ CUDA installation complete!
Cached files in: /content/drive/MyDrive/colab_cache

PYTORCH INSTALLATION
PyTorch: 2.1.0+cu118
CUDA available: True
CUDA version: 11.8
NumPy: 1.24.0


In [ ]:
# Install pointops that is needed for Sonata decoder head. This you have to run twice

%cd Pointcept/libs/pointops
!pip install -v -e.
%cd /content/drive/MyDrive/Pointcept


/content/drive/MyDrive/Pointcept/Pointcept/libs/pointops
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/drive/MyDrive/Pointcept/Pointcept/libs/pointops
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-4cqj4mfj/pointops.egg-info
  writing /tmp/pip-pip-egg-info-4cqj4mfj/pointops.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-4cqj4mfj/pointops.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-4cqj4mfj/pointops.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-4cqj4mfj/pointops.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-4cqj4mfj/pointops.egg-info/SOURCES.txt'
  /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:502: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
    warnin

In [ ]:
 #This install mostly installs depencencies having to do with pointops
 %cd /content/drive/MyDrive/Pointcept/Installs
 exec(open('/content/drive/MyDrive/Pointcept/Installs/setup_build_env.py').read())

/content/drive/MyDrive/Pointcept/Installs
2.1.0+cu118
NumPy: 1.24.0
NumPy: 1.24.0
CHECKING ENVIRONMENT
✓ GPU Available: NVIDIA A100-SXM4-40GB
✓ CUDA Version: 11.8
✓ PyTorch Version: 2.1.0+cu118

✓ CUDA Version for installations: cu118
FIXING NUMPY COMPATIBILITY
✓ NumPy 1.24.0 compatibility fixes applied
INSTALLING DEPENDENCIES


<string>:89: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
<string>:89: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
<string>:89: FutureWarning: In the future `np.str` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.


Installing basic dependencies...

Installing PyTorch Geometric dependencies...
Installing torch-scatter...
Installing torch-sparse...
Installing torch-cluster...
Installing torch-geometric...

Installing spconv...
CLONING POINTCEPT
✓ Cloned Pointcept
✓ GPU: NVIDIA A100-SXM4-40GB, Using CUDA architecture: 8.0
BUILDING POINTOPS
Building pointops with pip...
✓ pointops imported successfully

BUILDING POINTGROUP_OPS
Building pointgroup_ops with pip...
✓ pointgroup_ops imported successfully

FINAL VERIFICATION
✓ torch (2.1.0+cu118)
✓ numpy (1.24.0)
✓ sklearn (1.3.2)
✓ torch_cluster (installed)
✓ pointops (installed)
✓ pointgroup_ops (installed)
✓ spconv (2.3.8)

✅ Installation completed successfully!

To use Pointcept, add these lines at the start of your code:
```python
import sys
sys.path.insert(0, '/content/Pointcept')
sys.path.insert(0, '/content/Pointcept/libs/pointops')
sys.path.insert(0, '/content/Pointcept/libs/pointgroup_ops')
```

QUICK FUNCTIONALITY TEST
✓ Created test tensor on 

In [ ]:
#Load and impliment pointops patches and wrapper that fixes ballqueary errors in pointops
%cd /content/drive/MyDrive/Pointcept/Installs
exec(open('/content/drive/MyDrive/Pointcept/Installs/fix_ballquery_wrapper.py').read())

/content/drive/MyDrive/Pointcept/Installs
FIXING BALLQUERY WRAPPER AFTER TORCH_GEOMETRIC INSTALL

1. CHECKING ENVIRONMENT AND DEPENDENCIES
----------------------------------------
  Python version: 3.11.13
  ✓ Python version OK
  PyTorch version: 2.1.0+cu118
  CUDA available: True
  CUDA version: 11.8
  GPU: NVIDIA A100-SXM4-40GB
  PyTorch built with CUDA: 11.8

  Checking core dependencies:
  ✓ numpy
  ✓ plotly
  ✓ trimesh
  ✓ psutil

  ✓ Pointcept found at: /content/Pointcept

  Checking pointgroup_ops:
  ✓ pointgroup_ops found at: /content/drive/MyDrive/Pointcept/libs/pointgroup_ops/pointgroup_ops.cpython-311-x86_64-linux-gnu.so
  ⚠ WARNING: Loading .so file directly
  ✓ Has ballquery_batch_p: <class 'builtin_function_or_method'>
  ✓ pointgroup_ops_cuda found at: /usr/local/lib/python3.11/dist-packages/pointgroup_ops_cuda.cpython-311-x86_64-linux-gnu.so
  ✓ pointops installed (optional)


2. FIXING MODEL IMPORTS
----------------------------------------

  Processing point_group_v1m2

The following runs predictions using Sonta and pointgroup and saves the predictions are NPZ files.

In [ ]:
# Set your config and checkpoint paths here:
#Output path:
SCANNET_VAL_PATH = '/content/drive/MyDrive/Scannet_dataset/val'
OUTPUT_PATH = '/content/drive/MyDrive/Pointcept/scannet_val_evaluation_v1_epoch795_v02' #best epoch 465
#Checkpoint here:
checkpoint_path = '/content/drive/MyDrive/Pointcept/exp/scannet/insseg-pointgroup-sonata_config_4/model/epoch_795.pth'
#Change config file here:
cfg = Config.fromfile('/content/drive/MyDrive/Pointcept/Pointcept/configs/scannet/insseg-pointgroup-v1m2-0-ptv3-base_2.py')


# Create output directory
os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_PATH, 'predictions'), exist_ok=True)

print("="*60)
print("RUN INFERENCE AND SAVE PREDICTIONS WITH POINTGROUP + SONATA")
print("="*60)


print("\nLoading SONATA+PointGroup model...")
model = build_model(cfg.model).cuda()
model.eval()

#Loading checkpoint here
checkpoint = torch.load(checkpoint_path, map_location='cuda')
weight = OrderedDict()
for key, value in checkpoint["state_dict"].items():
    if key.startswith("module."):
        key = key[7:]
    weight[key] = value
model.load_state_dict(weight, strict=True)
print("Model loaded")


transform_list = [
    dict(type="CenterShift", apply_z=True),
    dict(
        type="Copy",
        keys_dict={
            "coord": "origin_coord",
            "segment": "origin_segment",
            "instance": "origin_instance",
        },
    ),
    dict(
        type="GridSample",
        grid_size=0.0175, #Experiment with different gridsizes usually from .01 - .025  smaller the size the more detail and vram useage
        hash_type="fnv",
        mode="train",
        return_grid_coord=True,
    ),
    dict(type="CenterShift", apply_z=False),
    dict(type="NormalizeColor"),
    dict(
        type="InstanceParser",
        segment_ignore_index=(-1, 0, 1),
        instance_ignore_index=-1,
    ),
]
transform = Compose(transform_list)

#Load validation scenes
val_scenes = sorted([d for d in os.listdir(SCANNET_VAL_PATH) if os.path.isdir(os.path.join(SCANNET_VAL_PATH, d))])
print(f"\nProcessing {len(val_scenes)} validation scenes...")

# Track stats
stats = {
    'processed': 0,
    'failed': 0,
    'total_predictions': 0,
    'total_gt_instances': 0
}

# Process each scene in folder
for scene_name in tqdm(val_scenes, desc="Running inference"):
    scene_path = os.path.join(SCANNET_VAL_PATH, scene_name)

    try:
        #Load scene data
        coord = np.load(os.path.join(scene_path, 'coord.npy')).astype(np.float32)
        color = np.load(os.path.join(scene_path, 'color.npy')).astype(np.float32)
        normal = np.load(os.path.join(scene_path, 'normal.npy')).astype(np.float32)
        gt_segment = np.load(os.path.join(scene_path, 'segment20.npy')).astype(np.int32)
        gt_instance = np.load(os.path.join(scene_path, 'instance.npy')).astype(np.int32)

        # Store originals
        num_points_original = len(coord)

        # Prepare data into a dictionary
        data_dict = {
            'coord': coord,
            'color': color,
            'normal': normal,
            'segment': gt_segment,
            'instance': gt_instance,
        }


        data_dict = transform(data_dict)

        #Convert to tensors
        for key in data_dict.keys():
            if isinstance(data_dict[key], np.ndarray):
                if key in ['segment', 'instance', 'grid_coord', 'origin_segment', 'origin_instance']:
                    data_dict[key] = torch.from_numpy(data_dict[key]).long()
                elif key == 'bbox':
                    data_dict[key] = torch.from_numpy(data_dict[key]).long()
                else:
                    data_dict[key] = torch.from_numpy(data_dict[key]).float()


        for key in data_dict.keys():
            if isinstance(data_dict[key], torch.Tensor):
                data_dict[key] = data_dict[key].cuda()


        feat = torch.cat([data_dict["coord"], data_dict["color"], data_dict["normal"]], dim=1)
        data_dict["feat"] = feat


        num_points = len(data_dict["coord"])
        data_dict["batch"] = torch.zeros(num_points, dtype=torch.long).cuda()
        data_dict["offset"] = torch.tensor([num_points], dtype=torch.long).cuda()

        if "origin_coord" in data_dict:
            origin_num_points = len(data_dict["origin_coord"])
            data_dict["origin_offset"] = torch.tensor([origin_num_points], dtype=torch.long).cuda()

        # Run inference
        with torch.no_grad():
            output_dict = model(data_dict)

        # Get predictions
        pred_masks = output_dict['pred_masks']
        pred_scores = output_dict['pred_scores']
        pred_classes = output_dict['pred_classes']

        # Map back to original points if voxelized
        if "origin_coord" in data_dict and pointops is not None:
            reverse, _ = pointops.knn_query(
                1,
                data_dict["coord"].float(),
                data_dict["offset"].int(),
                data_dict["origin_coord"].float(),
                data_dict["origin_offset"].int(),
            )
            reverse = reverse.cpu().flatten().long()
            pred_masks = pred_masks[:, reverse]

        #Converting to numpy
        pred_masks = pred_masks.cpu().numpy()
        pred_scores = pred_scores.cpu().numpy()
        pred_classes = pred_classes.cpu().numpy()

        #Make sure predictions match original point cloud size
        if pred_masks.shape[1] != num_points_original:
            print(f"\nWarning: {scene_name} - prediction size mismatch!")
            print(f"  Original points: {num_points_original}")
            print(f"  Prediction points: {pred_masks.shape[1]}")

        #Save scene predictions
        scene_output = os.path.join(OUTPUT_PATH, 'predictions', f'{scene_name}.npz')
        np.savez_compressed(
            scene_output,
            pred_masks=pred_masks,
            pred_scores=pred_scores,
            pred_classes=pred_classes,
            gt_segment=gt_segment,
            gt_instance=gt_instance,
            num_points=num_points_original
        )

        # Update stats
        stats['processed'] += 1
        stats['total_predictions'] += len(pred_scores)
        stats['total_gt_instances'] += len(np.unique(gt_instance[gt_instance >= 0]))

    except Exception as e:
        print(f"\nError in {scene_name}: {str(e)}")
        stats['failed'] += 1
        continue

# Save summary
summary = {
    'dataset': 'ScanNet validation',
    'model': 'SONATA+PointGroup',
    'checkpoint': checkpoint_path,
    'num_scenes': len(val_scenes),
    'processed_scenes': stats['processed'],
    'failed_scenes': stats['failed'],
    'total_predictions': stats['total_predictions'],
    'total_gt_instances': stats['total_gt_instances'],
    'avg_predictions_per_scene': stats['total_predictions'] / max(stats['processed'], 1),
    'avg_gt_instances_per_scene': stats['total_gt_instances'] / max(stats['processed'], 1),
    'output_path': OUTPUT_PATH
}



print("\n" + "="*60)
print("INFERENCE COMPLETE")
print(f"Processed: {stats['processed']}/{len(val_scenes)} scenes")
print(f"Failed: {stats['failed']} scenes")
print(f"Average predictions per scene: {summary['avg_predictions_per_scene']:.1f}")
print(f"Average GT instances per scene: {summary['avg_gt_instances_per_scene']:.1f}")
print(f"\nPredictions saved to: {OUTPUT_PATH}")
print("="*60)
print("\nNext: Run evaluation script on these predictions")

RUN INFERENCE AND SAVE PREDICTIONS WITH POINTGROUP + SONATA

Loading SONATA+PointGroup model...
Model loaded

Processing 312 validation scenes...


Running inference: 100%|██████████| 312/312 [32:44<00:00,  6.30s/it]



INFERENCE COMPLETE
Processed: 312/312 scenes
Failed: 0 scenes
Average predictions per scene: 21.7
Average GT instances per scene: 33.2

Predictions saved to: /content/drive/MyDrive/Pointcept/scannet_val_evaluation_v1_epoch795_v02

Next: Run evaluation script on these predictions


Pointgroup Evaluator un modified (automatically loads the most recent NPY files). This is where the actual evaluation happens.


In [ ]:


sys.path.insert(0, '/content/drive/MyDrive/Pointcept')


from pointcept.engines.hooks.evaluator import InsSegEvaluator

# Configuration
PRED_PATH = '/content/drive/MyDrive/Pointcept/scannet_val_evaluation_v1_epoch795_v20'

print("="*60)
print("EVALUATING WITH POINTCEPT'S OFFICIAL EVALUATOR")
print("="*60)

#Initialize evaluator
evaluator = InsSegEvaluator(
    segment_ignore_index=(-1, 0, 1),
    instance_ignore_index=-1
)

# Declare scannet 20 names (excluding wall and floor)
class_names = [
    "cabinet", "bed", "chair", "sofa", "table", "door",
    "window", "bookshelf", "picture", "counter", "desk",
    "curtain", "refrigerator", "shower curtain", "toilet",
    "sink", "bathtub", "otherfurniture"
]
evaluator.valid_class_names = class_names

#Load all predictions and prepare scenes
pred_files = sorted(os.listdir(os.path.join(PRED_PATH, 'predictions')))
print(f"Found {len(pred_files)} prediction files")

scenes = []
for pred_file in tqdm(pred_files, desc="Processing scenes"):
    scene_name = pred_file.replace('.npz', '')

    # Load data
    pred_data = np.load(os.path.join(PRED_PATH, 'predictions', pred_file))

    #Create prediction dict in the format evaluator expects
    pred_dict = {
        "pred_masks": pred_data['pred_masks'],
        "pred_scores": pred_data['pred_scores'],
        "pred_classes": pred_data['pred_classes']
    }

    #Get ground truth
    segment = pred_data['gt_segment']
    instance = pred_data['gt_instance']

    #Convert to torch tensors for associate_instances
    pred_dict_torch = {
        "pred_masks": torch.from_numpy(pred_dict["pred_masks"]),
        "pred_scores": torch.from_numpy(pred_dict["pred_scores"]),
        "pred_classes": torch.from_numpy(pred_dict["pred_classes"])
    }
    segment_torch = torch.from_numpy(segment)
    instance_torch = torch.from_numpy(instance)


    class MockTrainer:
        def __init__(self):
            self.cfg = type('cfg', (), {})()
            self.cfg.data = type('data', (), {})()
            self.cfg.data.num_classes = 20
            self.cfg.data.names = [
                "wall", "floor", "cabinet", "bed", "chair", "sofa", "table", "door",
                "window", "bookshelf", "picture", "counter", "desk", "curtain",
                "refrigerator", "shower curtain", "toilet", "sink", "bathtub", "otherfurniture"
            ]

    evaluator.trainer = MockTrainer()


    gt_instances, pred_instances = evaluator.associate_instances(
        pred_dict_torch, segment_torch, instance_torch
    )

    # Add to scenes
    scenes.append({
        "gt": gt_instances,
        "pred": pred_instances
    })

print(f"\nProcessed {len(scenes)} scenes")
print("Running evaluation...")

#Evaluate matches
ap_scores = evaluator.evaluate_matches(scenes)

#Display results
print("\n" + "="*60)
print("POINTCEPT EVALUATION RESULTS")
print("="*60)

all_ap = ap_scores["all_ap"]
all_ap_50 = ap_scores["all_ap_50%"]
all_ap_25 = ap_scores["all_ap_25%"]

print(f"\nOverall Performance:")
print(f"  mAP: {all_ap:.3f}")
print(f"  mAP@0.5: {all_ap_50:.3f}")
print(f"  mAP@0.25: {all_ap_25:.3f}")

print(f"\nPer-Class AP@0.5:")
print(f"{'Class':<20} {'AP':<10} {'AP@0.5':<10} {'AP@0.25':<10}")
print("-" * 50)

for label_name in evaluator.valid_class_names:
    if label_name in ap_scores["classes"]:
        ap = ap_scores["classes"][label_name]["ap"]
        ap_50 = ap_scores["classes"][label_name]["ap50%"]
        ap_25 = ap_scores["classes"][label_name]["ap25%"]
        print(f"{label_name:<20} {ap:<10.3f} {ap_50:<10.3f} {ap_25:<10.3f}")

print("="*60)


EVALUATING WITH POINTCEPT'S OFFICIAL EVALUATOR
Found 312 prediction files


Processing scenes: 100%|██████████| 312/312 [04:24<00:00,  1.18it/s]



Processed 312 scenes
Running evaluation...

POINTCEPT EVALUATION RESULTS

Overall Performance:
  mAP: 0.423
  mAP@0.5: 0.636
  mAP@0.25: 0.784

Per-Class AP@0.5:
Class                AP         AP@0.5     AP@0.25   
--------------------------------------------------
cabinet              0.327      0.562      0.758     
bed                  0.346      0.737      0.836     
chair                0.688      0.829      0.889     
sofa                 0.345      0.586      0.799     
table                0.436      0.684      0.792     
door                 0.289      0.532      0.647     
window               0.312      0.533      0.773     
bookshelf            0.286      0.576      0.753     
picture              0.417      0.576      0.668     
counter              0.067      0.250      0.645     
desk                 0.132      0.387      0.778     
curtain              0.351      0.574      0.764     
refrigerator         0.529      0.703      0.718     
shower curtain       0.628    

Optional: clear gpu cache. If you find that the GPU is filling up and not resetting you can try to clear it with the following snippit

In [ ]:


#Clear GPU CACHE
torch.cuda.empty_cache()
print("Cleared GPU cache after model loading.")

# ... (your existing code) ...

# After inference, if you have large intermediate tensors that are no longer needed
try:
    with torch.no_grad():
        output_dict = model(data_dict)
    print("✓ Inference successful")
except Exception as e:
    print(f"Error: {e}")
    # Try Point object approach
    try:
        from pointcept.models.utils import Point
        point = Point(
            coord=data_dict.get("coord"),
            feat=data_dict.get("feat"),
            batch=data_dict.get("batch"),
            grid_coord=data_dict.get("grid_coord"),
        )
        with torch.no_grad():
            output_dict = model(point)
        print("✓ Inference successful with Point object")
    except Exception as e2:
        print(f"Both approaches failed: {e2}")
        raise

torch.cuda.empty_cache()
print("Cleared GPU cache after inference.")

print("\nModel outputs:", output_dict.keys())



✓ Loaded SONATA+PointGroup model
Cleared GPU cache after model loading.
✓ Inference successful
Cleared GPU cache after inference.

Model outputs: dict_keys(['loss', 'seg_loss', 'bias_l1_loss', 'bias_cosine_loss', 'pred_scores', 'pred_masks', 'pred_classes'])
